# Recursive URL Loader
Web上のドキュメントをクローリングし、ローカルに保存します。  
データセットの作成に使用します。

In [12]:
# セットアップ
%pip install langchain-community
%pip install bs4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# データセットを作成する
csv形式のデータセットを作成します。

### データセットのサンプル

```csv
source,title,description,content,language,docs_updated_at
https://python.langchain.com/docs/integrations/document_loaders/,Document loaders | 🦜️🔗 Langchain,,"🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain ... Facebook",en,
```

In [1]:
# 定数の値を変更することで、再帰的に取得するページ数や保存するCSVファイル名を変更できる
URL_ROOT = "https://python.langchain.com/docs/integrations/document_loaders/"
MAX_DEPTH = 2
SUB_DIRECTORY = "recursive_url_loader/"
FILENAME_PREFIX = "langchain_docs"


# CSVファイルに結果を保存するためのユーティリティ
import os
from datetime import datetime
import csv

print(os.getcwd())

def save_docs_to_csv(
    csv_body: list[dict], sub_directory: str = "", filename_prefix: str = ""
) -> None:

    # Create directory
    directory = os.path.join(os.getcwd(), f"datasets/{sub_directory}")
    os.makedirs(directory, exist_ok=True)

    # Create CSV file name by timestamp
    csv_path = os.path.join(
        directory,
        f"{filename_prefix + '_' if filename_prefix != '' else ''}{datetime.now().strftime('%Y%m%d%H%M%S')}.csv",
    )

    with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
        # write header
        fieldname = csv_body[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldname)
        writer.writeheader()

        # write body
        for csv_row in csv_body:
            writer.writerow(csv_row)


# URL配下のドキュメントを再起的に取得しCSVファイルに保存する
from langchain_community.document_loaders.recursive_url_loader import (
    RecursiveUrlLoader,
    Document,
)
from bs4 import BeautifulSoup as Soup


def load_docs(url: str, max_depth: int) -> list[Document]:
    loader = RecursiveUrlLoader(
        url=url,
        max_depth=max_depth,
        extractor=lambda x: (lambda y: y.get_text() if y else None)(
            Soup(x, "html.parser").find(class_="theme-doc-markdown markdown")
        ),
    )
    docs = loader.load()
    return docs


def transform_docs_to_csv_body(docs: list[Document]) -> list[dict]:
    csv_body = [
        {
            "source": doc.metadata["source"],
            "title": doc.metadata["title"],
            "description": doc.metadata.get("description", ""),
            "content": doc.page_content.replace("\n", "\\n"),
            "language": doc.metadata["language"],
            "docs_updated_at": doc.metadata.get("docs_updated_at", ""),
        }
        for doc in docs
    ]
    return csv_body


def main() -> None:
    docs = load_docs(url=URL_ROOT, max_depth=MAX_DEPTH)
    print(f"ページ数:{len(docs)}")
    csv_body = transform_docs_to_csv_body(docs)
    save_docs_to_csv(
        csv_body=csv_body, sub_directory=SUB_DIRECTORY, filename_prefix=FILENAME_PREFIX
    )
    print("CSVファイルを出力しました")


if __name__ == "__main__":
    main()

/Users/kazuhisafukuda/dev/kairu-kun/src/notebook
ページ数:158
CSVファイルを出力しました


# recursive_url_loaderの動作確認
recursive_url_loader によってどんな値が取得できるかチェックするためのコードです。

### 参考:メタデータの形式

```python
{
    'source': 'https://python.langchain.com/docs/integrations/document_loaders/dropbox',
    'title': 'Dropbox | 🦜️🔗 Langchain',
    'description': 'Dropbox is a file hosting',
    'language': 'en'
}
```

In [ ]:
# recursive_url_loader によって再起的にコンテンツを取得できているかチェックする
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup

url = "https://python.langchain.com/docs/integrations/document_loaders/"
loader = RecursiveUrlLoader(
    url=url, max_depth=3, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

print(f"ページ数:{len(docs)}")
print(f"テキスト:" + docs[-1].page_content[:50].replace("\n", " "))
print(f"メタデータ:{docs[-1].metadata}")